In [ ]:
!pip install optuna

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import os
import pickle

import optuna
from optuna.trial import TrialState

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision import transforms

# Prepare Data

In [ ]:
# Load dataset
df = pd.read_csv("erdos_renyi_a-g_with_graph_features.csv")

In [ ]:
# Set list of predictors
predictors = ['p', 'Number of vertices', 'Number of edges', 'Edge density',
       'Mean degree', 'Standard deviation of degrees', 'Skewness of degrees',
       'Minimum degree', 'Maximum degree', 'Diameter', 'Radius',
       'Vertex connectivity', 'Edge connectivity',
       'Global clustering coefficient', 'Mean local clustering coefficient',
       'Standard deviation of local clustering coefficients',
       'Skewness of local clustering coefficients',
       'Minimum local clustering coefficient',
       'Maximum local clustering coefficient', 'Treewidth',
       'Average path length', 'Circuit rank', 'Girth',
       'Mean betweenness centrality',
       'Standard deviation of betweenness centralities',
       'Skewness of betweenness centralities',
       'Minimum betweenness centrality', 'Maximum betweenness centrality',
       'Algebraic connectivity', 'Von Neumann entropy',
       'Adjacency spectrum mean', 'Adjacency spectrum standard deviation',
       'Adjacency spectrum skewness', 'Adjacency spectrum min',
       'Adjacency spectrum max', 'Laplacian spectrum mean',
       'Laplacian spectrum standard deviation', 'Laplacian spectrum skewness',
       'Laplacian spectrum min', 'Laplacian spectrum max', 'Planarity',
       'Mean harmonic centrality',
       'Standard deviation of harmonic centralities',
       'Skewness of harmonic centralities', 'Minimum harmonic centrality',
       'Maximum harmonic centrality', 'Harmonic diameter', 'Mean core number',
       'Standard deviation of core numbers', 'Skewness of core numbers',
       'Minimum core number', 'Maximum core number', 'Chordality',
       'Haemers bound', 'Claw-free']

In [ ]:
# Split df into training & testing dataset (ratio of 80:20)
df_y = df['f_calls']
df_x = df[predictors]
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=.2)

In [ ]:
# Construct tensors for training & testing datasets
train_X = torch.tensor(train_x.to_numpy(), dtype=torch.float32)
train_Y = torch.tensor(train_y.to_numpy(), dtype=torch.float32).reshape(-1, 1)
test_X = torch.tensor(test_x.to_numpy(), dtype=torch.float32)
test_Y = torch.tensor(test_y.to_numpy(), dtype=torch.float32).reshape(-1, 1)

# Optuna Implementation

In [ ]:
DEVICE = torch.device("cpu")
EPOCHS = 1500

# We optimize the number of layers & hidden units
def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 5)
    layers = []

    # Set input & output dimensions
    in_features = len(predictors)
    output_size = 1

    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())

        in_features = out_features

    layers.append(nn.Linear(in_features, output_size))
    layers.append(nn.ReLU())

    return nn.Sequential(*layers)

# We optimize the optimizer type & learning rate
def objective(trial):
    # Generate model
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Construct tensors for training & testing datasets
    train_X = torch.tensor(train_x.to_numpy(), dtype=torch.float32)
    train_Y = torch.tensor(train_y.to_numpy(), dtype=torch.float32).reshape(-1, 1)
    test_X = torch.tensor(test_x.to_numpy(), dtype=torch.float32)
    test_Y = torch.tensor(test_y.to_numpy(), dtype=torch.float32).reshape(-1, 1)

    # Model training
    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()
        output = model(train_X)
        loss = F.mse_loss(output, train_Y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # Model validation
        model.eval()
        with torch.no_grad():
            output = model(test_X)

        r_2 = r2_score(np.array(test_Y.detach().numpy()), output.detach().numpy())

        trial.report(r_2, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return r_2

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))